In [1]:
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch

import math

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from models.Update import LocalUpdate
from models.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2
from models.Fed import FedAvg
from models.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models.test import test_img

%load_ext autoreload
%autoreload 2

import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from models.Update import LocalUpdate
from models.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2, CNNMnist3
from models.Fed import FedAvg
from models.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models.test import test_img
from torch.utils.data import DataLoader, Dataset
from utils.functions import *

%load_ext autoreload
%autoreload 2

class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label

class my_argument:    
    epochs = 200    #"rounds of training"
    num_users = 15  # "number of users: N"
    num_partition = 6 # "number of users: K"
    frac = 0.5 #"the fraction of clients: C"
    local_ep = 1 #"the number of local epochs: E"
    local_bs = 200 #"local batch size: B"
    bs=200 #"test batch size"
    lr=0.01 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"
    opt='ADAM'
    loss='Custom' # 'Custom' or 'Default'

    # model arguments
    model = 'cnn'
    kernel_num=9 #, help='number of each kind of kernel')
    kernel_sizes='3,4,5' #  help='comma-separated kernel size to use for convolution')
    norm='None' #, help="batch_norm, layer_norm, or None")
    num_filters=32 #, help="number of filters for conv nets")
    max_pool='True' #help="Whether use max pooling rather than strided convolutions")

    # other arguments
    dataset='mnist' #, help="name of dataset")
    iid=1
    num_classes=10#, help="number of classes")
    num_channels=1#, help="number of channels of imges")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose='False'#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    
args = my_argument()

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')

# load dataset and split users
trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset_train = datasets.MNIST('../data/mnist/', train=True, download=True, transform=trans_mnist)
dataset_test = datasets.MNIST('../data/mnist/', train=False, download=True, transform=trans_mnist)

dict_users = mnist_iid(dataset_train, args.num_partition)

encoding_input_array_np = np.empty((len(dataset_train),28*28))
encoding_label_array_np = np.empty((len(dataset_train),args.num_classes))
print("size of X:" ,encoding_input_array_np.shape)
print("size of Y:" ,encoding_label_array_np.shape)

Size_submatrices = int(60000/args.num_partition)

for i in range(args.num_partition):
    
    stt_pos = i*Size_submatrices
    end_pos = (i+1)*Size_submatrices
#     print(i,stt_pos,end_pos)
    Temp_train = DataLoader(DatasetSplit(dataset_train, dict_users[i]), batch_size=Size_submatrices, shuffle=True)
    
    for batch_idx, (images, labels) in enumerate(Temp_train):
        
        images_np = images.detach().cpu().numpy()
        encoding_input_array_np[stt_pos:end_pos,:] = np.reshape(images_np, (Size_submatrices,28*28))
#         print(encoding_input_array_np[stt_pos:end_pos,:].shape)

        onehot_labels = torch.nn.functional.one_hot(labels,num_classes=args.num_classes)
        labels_np = onehot_labels.detach().cpu().numpy()
#         print(labels_np.shape)
        encoding_label_array_np[stt_pos:end_pos,:] = labels_np


# print(labels_np[0:10,:])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
size of X: (60000, 784)
size of Y: (60000, 10)


In [7]:
from models.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist1, CNNMnist2, CNNMnist3

from models.Update import LocalUpdate_with_BACC
from models.Fed import FedAvg_with_BACC_Dec, FedAvg_with_LCC_Dec

from utils.functions import *
import math

K = 6
T = 0
sigma = 1
Noise_Alloc = []

Signal_Alloc = []
for i in range(K+T):
    if i not in Noise_Alloc:
        Signal_Alloc.append(i)


j_array = np.array(range(K+T))
alpha_array = np.cos((2*j_array+1)*math.pi/(2*(K+T))) #np.cos((2*j_array+1)*math.pi/(2*K))

# alpha_array = np.array([-5.87785252e-01, 5.87785252e-01])
# print("alpha_array: ",alpha_array,'\n')

B = 0.5


N_array = [6,15]
alloc_case = 3
B_array = [0.5]

N_trials = 1
N_epochs = 30



loss_test_arr_v0_0 = np.zeros((len(N_array),len(B_array),N_trials,N_epochs))
acc_test_arr_v0_0  = np.zeros((len(N_array),len(B_array),N_trials,N_epochs))

for N_idx in range(len(N_array)):
    
    N = N_array[N_idx]
    
    
           
        
    # print("alpha_array: ",alpha_array,'\n')
    
    
    for B_idx in range(len(B_array)):
        
        B = B_array[B_idx]
        z_array = []
#         while(len(z_array)<N):
#             z_tmp = np.random.uniform(-1,1,1)
#             MIS_tmp = MutualInformationSecurity(alpha_array[Signal_Alloc], alpha_array[Noise_Alloc],[z_tmp], 1,sigma)
#             if MIS_tmp < B and MIS_tmp > 0.1:
#                 z_array.append(z_tmp[0])
#         
#         z_array = np.sort(z_array)
        print(N_idx)
        i_array = np.array(range(N))
        z_array = np.cos(i_array*2*math.pi/N/2) # np.cos(i_array*2*math.pi/N/2)
#         if N==2:
#             z_array = np.array([-0.88, 0.88])
#         elif N ==4:
#             z_array = np.array([-0.88, -0.25, 0.25, 0.88])
#         elif N ==5:
#             z_array = np.array([-0.88, -0.25, -0.2, 0.25, 0.88])
#         elif N ==6:
#             z_array = np.array([-0.94, -0.88, -0.25, 0.25, 0.88, 0.94])
#         elif N ==7:
#             z_array = np.array([-0.94, -0.88, -0.25, 0, 0.25, 0.88, 0.94])
#         else:
#             z_array = np.array([-0.94, -0.88, -0.25, -0.2, 0.2, 0.25, 0.88, 0.94])

            
        print('z_array:',z_array)        
        
        _Noise_label = np.ones((30000*T,10)) * 0.1

        X_tilde,a,b = BACC_Enc_Data_v3(encoding_input_array_np, N, K, T, sigma, alpha_array, z_array, _Noise_Alloc = Noise_Alloc, _is_LCC=False) #BACC_Enc(encoding_input_array_np, alpha_array, z_array)
        y_tilde,a,b = BACC_Enc_Data_v3(encoding_label_array_np, N, K, T, sigma, alpha_array, z_array, _Noise_Alloc = Noise_Alloc, _Noise = _Noise_label, is_predefined_noise=True, _is_LCC=False) #BACC_Enc(encoding_label_array_np, alpha_array, z_array)

        m = N # m is the number of received result @ master
    #     print('number of results:',m)

        for trial_idx in range(N_trials):
            print('(T, sigma)=',T,sigma,') ',trial_idx,'-th Trial!!')

            net_glob = CNNMnist1(args=args)
            net_glob.cuda()
            net_glob.train()

            # copy weights
            w_glob = net_glob.state_dict()

            for iter in range(N_epochs): #args.epochs
                w_locals, loss_locals = [], []
                idxs_users = np.random.choice(range(N), m, replace=False)
                idxs_users = np.sort(idxs_users)
                print('selected users:',idxs_users)
                
#                 coded_net = BACC_Enc_Model_withNoise_v3(net_glob.cuda(), N, K, T, 0, alpha_array, z_array, _Noise_Alloc=Noise_Alloc, _is_LCC=False)

                dec_z_array = []
                for idx in idxs_users: #for idx in range(N):
            #         print(idx)
                    local = LocalUpdate_with_BACC(args=args, dataset=X_tilde[idx,:,:], label=y_tilde[idx,:,:])
#                     w, loss = local.train(net=copy.deepcopy(coded_net[idx]).cuda())
                    w, loss = local.train(net=copy.deepcopy(net_glob).cuda())
                    w_locals.append(copy.deepcopy(w))
                    loss_locals.append(copy.deepcopy(loss))

                    dec_z_array.append(z_array[idx])


                # update global weights
                
                w_glob = FedAvg_with_BACC_Dec(w_locals, alpha_array[Signal_Alloc], dec_z_array)
#                 w_glob = FedAvg_with_LCC_Dec(w_locals, alpha_array[Signal_Alloc], dec_z_array)

                # copy weight to net_glob
                net_glob.load_state_dict(w_glob)

                # print loss
            #     acc_train, loss_train = test_img(net_glob, dataset_train, args)

            #     loss_train_arr.append(loss_train)

                acc_test, loss_test = test_img(net_glob, dataset_test, args)
                acc_test_arr_v0_0[N_idx][B_idx][trial_idx][iter] = acc_test
                loss_test_arr_v0_0[N_idx][B_idx][trial_idx][iter] = loss_test
                if iter % 1 ==0:
                    print('Round {:3d}, Average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_test, acc_test))
                #print(loss_train)

0
z_array: [ 1.00000000e+00  8.66025404e-01  5.00000000e-01  6.12323400e-17
 -5.00000000e-01 -8.66025404e-01]
@BACC_Enc: N,K,T, m_i= 6 6 0 10000 

@BACC_Enc: N,K,T, m_i= 6 6 0 10000 

(T, sigma)= 0 1 )  0 -th Trial!!
selected users: [0 1 2 3 4 5]

Test set: Average loss: 2.3026 
Accuracy: 980/10000 (9.80%)

Round   0, Average loss 2.303 Test accuracy 9.800
selected users: [0 1 2 3 4 5]

Test set: Average loss: 2.3026 
Accuracy: 980/10000 (9.80%)

Round   1, Average loss 2.303 Test accuracy 9.800
selected users: [0 1 2 3 4 5]

Test set: Average loss: 2.3026 
Accuracy: 980/10000 (9.80%)

Round   2, Average loss 2.303 Test accuracy 9.800
selected users: [0 1 2 3 4 5]

Test set: Average loss: 2.3025 
Accuracy: 1028/10000 (10.28%)

Round   3, Average loss 2.303 Test accuracy 10.280
selected users: [0 1 2 3 4 5]

Test set: Average loss: 2.2646 
Accuracy: 3651/10000 (36.51%)

Round   4, Average loss 2.265 Test accuracy 36.510
selected users: [0 1 2 3 4 5]

Test set: Average loss: 1.4479 
Accu


Test set: Average loss: 1.9916 
Accuracy: 9395/10000 (93.95%)

Round  19, Average loss 1.992 Test accuracy 93.950
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 2.0030 
Accuracy: 9407/10000 (94.07%)

Round  20, Average loss 2.003 Test accuracy 94.070
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 1.9509 
Accuracy: 9387/10000 (93.87%)

Round  21, Average loss 1.951 Test accuracy 93.870
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 1.9977 
Accuracy: 9421/10000 (94.21%)

Round  22, Average loss 1.998 Test accuracy 94.210
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 1.9841 
Accuracy: 9366/10000 (93.66%)

Round  23, Average loss 1.984 Test accuracy 93.660
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 1.9957 
Accuracy: 9407/10000 (94.07%)

Round  24, Average loss 1.996 Test accuracy 94

In [5]:
from models.Update import LocalUpdate_with_BACC
from models.Fed import FedAvg_with_BACC_Dec, FedAvg_with_LCC_Dec

from utils.functions import *
import math

K = 6
T = 0
sigma = 1
Noise_Alloc = []

Signal_Alloc = []
for i in range(K+T):
    if i not in Noise_Alloc:
        Signal_Alloc.append(i)


j_array = np.array(range(K+T))
alpha_array = np.cos((2*j_array+1)*math.pi/(2*(K+T))) #np.cos((2*j_array+1)*math.pi/(2*K))

# alpha_array = np.array([-5.87785252e-01, 5.87785252e-01])
# print("alpha_array: ",alpha_array,'\n')

B = 0.5


N_array = [6,15]
alloc_case = 3
B_array = [0.5]

N_trials = 1
N_epochs = 30



loss_test_arr_v0_1 = np.zeros((len(N_array),len(B_array),N_trials,N_epochs))
acc_test_arr_v0_1  = np.zeros((len(N_array),len(B_array),N_trials,N_epochs))

for N_idx in range(len(N_array)):
    
    N = N_array[N_idx]
    
    
           
        
    # print("alpha_array: ",alpha_array,'\n')
    
    
    for B_idx in range(len(B_array)):
        
        B = B_array[B_idx]
        z_array = []
#         while(len(z_array)<N):
#             z_tmp = np.random.uniform(-1,1,1)
#             MIS_tmp = MutualInformationSecurity(alpha_array[Signal_Alloc], alpha_array[Noise_Alloc],[z_tmp], 1,sigma)
#             if MIS_tmp < B and MIS_tmp > 0.1:
#                 z_array.append(z_tmp[0])
#         
#         z_array = np.sort(z_array)
        print(N_idx)
        i_array = np.array(range(N))
        z_array = np.cos(i_array*2*math.pi/N/2) # np.cos(i_array*2*math.pi/N/2)
#         if N==2:
#             z_array = np.array([-0.88, 0.88])
#         elif N ==4:
#             z_array = np.array([-0.88, -0.25, 0.25, 0.88])
#         elif N ==5:
#             z_array = np.array([-0.88, -0.25, -0.2, 0.25, 0.88])
#         elif N ==6:
#             z_array = np.array([-0.94, -0.88, -0.25, 0.25, 0.88, 0.94])
#         elif N ==7:
#             z_array = np.array([-0.94, -0.88, -0.25, 0, 0.25, 0.88, 0.94])
#         else:
#             z_array = np.array([-0.94, -0.88, -0.25, -0.2, 0.2, 0.25, 0.88, 0.94])

            
        print('z_array:',z_array)        
        
        _Noise_label = np.ones((30000*T,10)) * 0.1

        X_tilde,a,b = BACC_Enc_Data_v3(encoding_input_array_np, N, K, T, sigma, alpha_array, z_array, _Noise_Alloc = Noise_Alloc, _is_LCC=False) #BACC_Enc(encoding_input_array_np, alpha_array, z_array)
        y_tilde,a,b = BACC_Enc_Data_v3(encoding_label_array_np, N, K, T, sigma, alpha_array, z_array, _Noise_Alloc = Noise_Alloc, _Noise = _Noise_label, is_predefined_noise=True, _is_LCC=False) #BACC_Enc(encoding_label_array_np, alpha_array, z_array)

        m = N # m is the number of received result @ master
    #     print('number of results:',m)

        for trial_idx in range(N_trials):
            print('(T, sigma)=',T,sigma,') ',trial_idx,'-th Trial!!')

            net_glob = CNNMnist2(args=args)
            net_glob.cuda()
            net_glob.train()

            # copy weights
            w_glob = net_glob.state_dict()

            for iter in range(N_epochs): #args.epochs
                w_locals, loss_locals = [], []
                idxs_users = np.random.choice(range(N), m, replace=False)
                idxs_users = np.sort(idxs_users)
                print('selected users:',idxs_users)
                
#                 coded_net = BACC_Enc_Model_withNoise_v3(net_glob.cuda(), N, K, T, 0, alpha_array, z_array, _Noise_Alloc=Noise_Alloc, _is_LCC=False)

                dec_z_array = []
                for idx in idxs_users: #for idx in range(N):
            #         print(idx)
                    local = LocalUpdate_with_BACC(args=args, dataset=X_tilde[idx,:,:], label=y_tilde[idx,:,:])
#                     w, loss = local.train(net=copy.deepcopy(coded_net[idx]).cuda())
                    w, loss = local.train(net=copy.deepcopy(net_glob).cuda())
                    w_locals.append(copy.deepcopy(w))
                    loss_locals.append(copy.deepcopy(loss))

                    dec_z_array.append(z_array[idx])


                # update global weights
                
                w_glob = FedAvg_with_BACC_Dec(w_locals, alpha_array[Signal_Alloc], dec_z_array)
#                 w_glob = FedAvg_with_LCC_Dec(w_locals, alpha_array[Signal_Alloc], dec_z_array)

                # copy weight to net_glob
                net_glob.load_state_dict(w_glob)

                # print loss
            #     acc_train, loss_train = test_img(net_glob, dataset_train, args)

            #     loss_train_arr.append(loss_train)

                acc_test, loss_test = test_img(net_glob, dataset_test, args)
                acc_test_arr_v0_1[N_idx][B_idx][trial_idx][iter] = acc_test
                loss_test_arr_v0_1[N_idx][B_idx][trial_idx][iter] = loss_test
                if iter % 1 ==0:
                    print('Round {:3d}, Average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_test, acc_test))
                #print(loss_train)

0
z_array: [ 1.00000000e+00  8.66025404e-01  5.00000000e-01  6.12323400e-17
 -5.00000000e-01 -8.66025404e-01]
@BACC_Enc: N,K,T, m_i= 6 6 0 10000 

@BACC_Enc: N,K,T, m_i= 6 6 0 10000 

(T, sigma)= 0 1 )  0 -th Trial!!
selected users: [0 1 2 3 4 5]

Test set: Average loss: 2.3023 
Accuracy: 1115/10000 (11.15%)

Round   0, Average loss 2.302 Test accuracy 11.150
selected users: [0 1 2 3 4 5]

Test set: Average loss: 2.7549 
Accuracy: 7688/10000 (76.88%)

Round   1, Average loss 2.755 Test accuracy 76.880
selected users: [0 1 2 3 4 5]

Test set: Average loss: 25.4866 
Accuracy: 7437/10000 (74.37%)

Round   2, Average loss 25.487 Test accuracy 74.370
selected users: [0 1 2 3 4 5]

Test set: Average loss: 34.5410 
Accuracy: 8590/10000 (85.90%)

Round   3, Average loss 34.541 Test accuracy 85.900
selected users: [0 1 2 3 4 5]

Test set: Average loss: 96.0410 
Accuracy: 8472/10000 (84.72%)

Round   4, Average loss 96.041 Test accuracy 84.720
selected users: [0 1 2 3 4 5]

Test set: Average los


Test set: Average loss: 43.6339 
Accuracy: 7466/10000 (74.66%)

Round  21, Average loss 43.634 Test accuracy 74.660
selected users: [0 1 2 3 4 5 6 7 8]

Test set: Average loss: 49.6264 
Accuracy: 7499/10000 (74.99%)

Round  22, Average loss 49.626 Test accuracy 74.990
selected users: [0 1 2 3 4 5 6 7 8]

Test set: Average loss: 40.9937 
Accuracy: 7552/10000 (75.52%)

Round  23, Average loss 40.994 Test accuracy 75.520
selected users: [0 1 2 3 4 5 6 7 8]

Test set: Average loss: 43.1204 
Accuracy: 7575/10000 (75.75%)

Round  24, Average loss 43.120 Test accuracy 75.750
selected users: [0 1 2 3 4 5 6 7 8]

Test set: Average loss: 43.8685 
Accuracy: 7536/10000 (75.36%)

Round  25, Average loss 43.868 Test accuracy 75.360
selected users: [0 1 2 3 4 5 6 7 8]

Test set: Average loss: 55.5360 
Accuracy: 7223/10000 (72.23%)

Round  26, Average loss 55.536 Test accuracy 72.230
selected users: [0 1 2 3 4 5 6 7 8]

Test set: Average loss: 35.1130 
Accuracy: 8032/10000 (80.32%)

Round  27, Averag

KeyboardInterrupt: 

In [3]:
j_array = np.array(range(K+T))
alpha_array = np.cos((2*j_array+1)*math.pi/(2*(K+T))) #np.cos((2*j_array+1)*math.pi/(2*K))

i_array = np.array(range(N))
z_array = np.cos(i_array*2*math.pi/N/2) # np.cos(i_array*2*math.pi/N/2)

print(alpha_array)
print(z_array)

[ 0.96592583  0.70710678  0.25881905 -0.25881905 -0.70710678 -0.96592583]
[ 1.          0.9781476   0.91354546  0.80901699  0.66913061  0.5
  0.30901699  0.10452846 -0.10452846 -0.30901699 -0.5        -0.66913061
 -0.80901699 -0.91354546 -0.9781476 ]


In [4]:
from models.Update import LocalUpdate_with_BACC
from models.Fed import FedAvg_with_BACC_Dec

# training
loss_train_arr = []
loss_test_arr = []
acc_test_arr = []
net_best = None
best_loss = None

N_trials = 1
N_epochs = 30

# m_array = np.array(range(4,16)) # m is the number of received result @ master
m_array = np.array([15]) # m is the number of received result @ master
loss_test_arr = np.empty((len(m_array),N_trials,N_epochs))
acc_test_arr  = np.empty((len(m_array),N_trials,N_epochs))

for m_idx in range(len(m_array)):   
    
    m = m_array[m_idx] # m is the number of received result @ master
    print('number of results:',m)
    
    for trial_idx in range(N_trials):
        print('(m=',m,') ',trial_idx,'-th Trial!!')
        
        net_glob = CNNMnist2(args=args)
        net_glob.cuda()
        net_glob.train()

        # copy weights
        w_glob = net_glob.state_dict()

        for iter in range(N_epochs): #args.epochs
            w_locals, loss_locals = [], []
            idxs_users = np.random.choice(range(N), m, replace=False)
            idxs_users = np.sort(idxs_users)
            print('selected users:',idxs_users)

            dec_z_array = []
            for idx in idxs_users: #for idx in range(N):
        #         print(idx)
                local = LocalUpdate_with_BACC(args=args, dataset=X_tilde[idx,:,:], label=y_tilde[idx,:,:])
                w, loss = local.train(net=copy.deepcopy(net_glob).cuda())
                w_locals.append(copy.deepcopy(w))
                loss_locals.append(copy.deepcopy(loss))

                dec_z_array.append(z_array[idx])


            # update global weights
            #w_glob = FedAvg(w_locals)
            w_glob = FedAvg_with_BACC_Dec(w_locals, alpha_array, dec_z_array)

            # copy weight to net_glob
            net_glob.load_state_dict(w_glob)

            # print loss
        #     acc_train, loss_train = test_img(net_glob, dataset_train, args)

        #     loss_train_arr.append(loss_train)

            acc_test, loss_test = test_img(net_glob, dataset_test, args)
            acc_test_arr[m_idx][trial_idx][iter] = acc_test
            loss_test_arr[m_idx][trial_idx][iter] = loss_test
            if iter % 1 ==0:
                print('Round {:3d}, Average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_test, acc_test))
            #print(loss_train)

number of results: 15
(m= 15 )  0 -th Trial!!
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 2.3026 
Accuracy: 980/10000 (9.80%)

Round   0, Average loss 2.303 Test accuracy 9.800
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 2.0779 
Accuracy: 5473/10000 (54.73%)

Round   1, Average loss 2.078 Test accuracy 54.730
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 0.2006 
Accuracy: 9508/10000 (95.08%)

Round   2, Average loss 0.201 Test accuracy 95.080
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 0.2194 
Accuracy: 9455/10000 (94.55%)

Round   3, Average loss 0.219 Test accuracy 94.550
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Test set: Average loss: 0.2028 
Accuracy: 9398/10000 (93.98%)

Round   4, Average loss 0.203 Test accuracy 93.980
selected users: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]

Tes